# GCC's ssc-cdi : ptychography pipeline for EMA v0.7.12
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/

## Imports

In [1]:
%matplotlib widget
import os, json, h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sscInstall
import sscCdi
import sscResolution
import time
from PIL import Image

print(f'sscCdi version: {sscCdi.__version__}')

sscCdi version: 0.6.40
sscResolution version: 0.6


## Inputs

In [2]:
input_dict = {
    
"beamline": "EMA",
"detector": "Moibipix",
    
"data_path": "/ibira/lnls/beamlines/ema/proposals/20210094/data/ptycho-files/ptycho_file_001.h5",
"beamline_parameters_path": "/ibira/lnls/beamlines/ema/proposals/20210094/data/ptycho-files/ptycho_aux.h5",
"output_path": "/ibira/lnls/beamlines/ema/proposals/20210094/proc/ptycho-files", # you must create the proc folder before run, or give a valid output path
 
    
"positions_unit_conversion": 1.0,
    
"flatfield": "/ibira/lnls/labs/ldet/mobipix_15D/mobipix_007/01_Calibracao/Flat Field/100imgs/Ge_FF.tif", # path to flat fiedl
"mask": "", # path to mask, must be in .tif

"CPUs": 32,
"GPUs": [0],

'binning':1,
    
"detector_distance": 2.9, # meters
    
"DP_center": [362,376], # [center_y, center_x]
"DP_radius": 100,  # crop radius

"position_rotation": 0.0,

"object_padding": 30,

"incoherent_modes": 1,
"probe_support": [ "circular", 100,0,0 ], # [ "cross", cross_width, border, center_width ], ["circular",radius]
# "probe_support": [ "cross", 47, 10, 94],

"fresnel_number": -0.001,

"initial_obj":  ["random"], 
"initial_probe": ['inverse'],
    
'Algorithm1': {'Batch': 64,
                'Beta': 0.995,
                'Epsilon': 0.01,
                'Iterations': 200,
                'Name': 'RAAR',
                'ProbeCycles': 4,
                'TV': 0},

'Algorithm2': {'Batch': 64,
                'Epsilon': 0.01,
                'Iterations': 100,
                'Name': 'positioncorrection',
                'ObjBeta': 0.97,
                'ProbeBeta': 0.95,
                'TV': 0.0001},

'Algorithm3': {'Batch': 64,
                'Epsilon': 0.01,
                'Iterations': 70,
                'Name': 'GL',
                'ObjBeta': 0.97,
                'ProbeBeta': 0.95,
                 'TV': 0.0001},

# 'Algorithm3': {'Batch': 64,
#                 'Epsilon': 0.1,
#                 'Iterations': 5,
#                 'Name': 'GL',
#                 'ObjBeta': 0.5,
#                 'ProbeBeta': 0.5,
#                 'TV': 0.01},
    
}

input_dict = sscCdi.ema.ema_ptycho_processing.define_paths(input_dict)

	Data path:  /ibira/lnls/labs/tepui/proposals/20210062/camila/ema/proposals/00000000/data/ptycho-files/ptycho_file_001.h5
	Output path: /ibira/lnls/labs/tepui/proposals/20210062/camila/ema/proposals/00000000/proc/ptycho-files


## Cropping diffraction patterns

In [ ]:
# getting diffraction patterns
data = h5py.File(input_dict["data_path"],'r')
diffraction_patterns = data["entry/data/data"][()]
diffraction_patterns = diffraction_patterns.astype(np.float32)

# appling mask and flatfield
flat_field = np.array(Image.open(input_dict["flatfield"])).astype(np.float32)
mask = np.array(Image.open(input_dict["mask"])).astype(np.float32)

for i in range(diffraction_patterns.shape[0]):
    diffraction_patterns[i,:,:] = diffraction_patterns[i,:,:]*flat_field
    diffraction_patterns[i,:,:][mask==1] = -1 # setting bad pixels as -1

# cropping data
diffraction_patterns_cropped = sscCdi.ema.crop_data(input_dict, diffraction_patterns)

# aditional masking
# diffraction_patterns_cropped[diffraction_patterns_cropped==0] = -1
diffraction_patterns_cropped[:,94:107,95:104] = -1

# visualization
sscCdi.misc.deploy_visualizer(diffraction_patterns_cropped, title='Sinogram',cmap='viridis', norm="LogNorm")

## Ptychography

In [3]:
# calling ptycho
obj, probe, input_dict = sscCdi.ema.ema_ptycho_processing.ema_ptychography(input_dict,diffraction_patterns_cropped)

# saving variables
sscCdi.misc.save_variable(input_dict, obj,name='object')
sscCdi.misc.save_variable(input_dict,probe,name='probe')
sscCdi.misc.save_json_logfile(input_dict) 

	Object pixel size = 0.16 nm
	Limit thickness for resolution of 1 pixel: 0.004 microns
Fresnel number: -0.001
Creating initial probe...
	Probe shape (512,512) with 1 incoherent mode(s)
Setting probe support...
Creating initial object...
Diffraction Patterns: (8, 512, 512)
Initial Object: (12914, 6264)
Initial Probe: (1, 512, 512)
Probe Support: (1, 512, 512)
Probe Positions: (8, 4)
Creating datapack...
Total datapack size: 0.66 GBs
Starting ptychography... using 1 GPUs [0] and 32 CPUs
	Calling 70 iterations of RAAR algorithm...
	Done in: 2.56 seconds
	Calling 50 iterations of positioncorrection algorithm...
	Done in: 1.82 seconds
	Calling 5 iterations of GL algorithm...
	Done in: 0.29 seconds


In [4]:
if 1: # phase
    preview = np.angle(obj)
else: # magnitude
    preview = np.abs(obj)

sscCdi.misc.deploy_visualizer(preview,title='Sinogram',cmap='gray')

## Unwrap

In [ ]:
phase = sscCdi.caterete.unwrap_in_parallel(preview)
# sscCdi.misc.save_variable(input_dict,phase, name = 'object_unwrapped')
sscCdi.misc.deploy_visualizer(phase,title='Sinogram',cmap='gray')

## Equalize

In [ ]:
dic = {}
dic["equalize_invert"] = True # invert phase shift signal from negative to positive
dic["equalize_ROI"] = [] # region of interest of null region around the sample used for phase ramp and offset corrections
dic["equalize_remove_phase_gradient"] = True  # if empty and equalize_ROI = [], will subtract best plane fit from whole image
dic["equalize_local_offset"] = False # remove offset of each frame from the mean of ROI 
dic["equalize_set_min_max"]= [] # [minimum,maximum] threshold values for whole volume
dic["equalize_non_negative"] = False # turn any remaining negative values to zero

frame = phase[0] # make sure to select a 2D frame
equalized_frame = sscCdi.equalize_frame(dic,frame)

figure, ax = plt.subplots(dpi=100)
ax.imshow(equalized_frame,cmap='gray')